# 실행하기전 주의사항: 실행할때마다 새창으로 이미지를 띄움



In [9]:

import numpy as np
import cv2


image = cv2.imread("outline.jpg")
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
gray = cv2.GaussianBlur(gray, (3, 3), 0)
cv2.imshow("Gray", gray)
cv2.waitKey(0)

#이미지를 로드하고 그레이 스케일로 변환하고 프리게 만듬

#여기서 단순히 이미지를 디스크에로드 한 다음 RGB (Red, Green, Blue) 색상 공간에서 그레이 스케일로 변환

#또한 고주파 노이즈를 줄이기 위해 이미지를 약간 흐리게 처리하고 

#나중에 물체를 찾는 데 사용 된 코드의 정확성을 높임



255

In [15]:
# 이미지 가장자리 즉,윤곽선(outline)을 감지

edged = cv2.Canny(gray, 10, 250)
cv2.imshow("Edged", edged)
cv2.waitKey(0)

#사물의 윤곽을 대부분 밝혀냄 

#하지만 일부 윤곽선이 깨끗하고 완벽히 처리되지않음.

13

In [11]:
#책을 성공적으로 감지하기 위해서는 우리가 필요로 하는 간격이있음

kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (7, 7))
closed = cv2.morphologyEx(edged, cv2.MORPH_CLOSE, kernel)
cv2.imshow("Closed", closed)
cv2.waitKey(0)

#이를 해결하기 위해 영상의 흰색 픽셀 사이의 간격을 좁히기 위해 "닫기" 작업,즉 틈새 색칠을 해야함

#이게 무슨말이냐면 글자나 기타 좁은 선안에있는 육관석의 틈새들을 
#색칠한다는 개념으로 알면 이해하기편함


255

In [12]:
#이번엔 전단계 오브젝트의 틈새 색칠의 잔처리를 끝난후의 진짜 외곽선을 감지해야함
#이것을 감지하기 위해서는 cv2.findContours함수를 사용함

(_, cnts, _) = cv2.findContours(closed.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
total = 0


#책은 직사각형이고 사각형에는 4 개의 꼭지점이 있음 
#따라서 윤곽을 검사하여 4 개의 정점이있는 것을 발견하면 이미지이고 
#아닌것중하나는 책이아니라는것을 판단 할 수 있다

#어떤 윤곽선이 책인지 아닌지 확인하려면 각 육관석을 개별적으로 반복해야함

In [13]:
# 윤곽선 반복
for c in cnts:
    # 대략적인 윤곽선
    peri = cv2.arcLength(c, True)
    approx = cv2.approxPolyDP(c, 0.02 * peri, True)

    # 정확한 윤곽선이 4점이면 다음과 같은 점을 가정함
    # 이 윤곽선은 책이고, 책은 직사각형이기 때문에 4개의 점이있다
    if len(approx) == 4:
        cv2.drawContours(image, [approx], -1, (0, 255, 0), 4)
        total += 1

In [16]:
# display the output
print "나는 {0}개의 책을 찾았어용 ㅋㅋ ".format(total)
cv2.imshow("Output", image)
cv2.waitKey(0)

나는 4개의 책을 찾았어용 ㅋㅋ 


255

직접 스크립트를 실행하려면 터미널을 열고 다음 명령을 실행
$ python find_books.py

<접근법 정리>
1.이미지를 디스크에서 로드하고 그레이 스케일로 변환
2.이미지를 약간 흐리게 만듬
3.Canny 엣지(edge)검출기를 사용하여 영상에 있는 가장자리(예:윤곽선)를 검
4.닫기 형태의 연산을 적용하여 외곽선의 간격을 좁힘
5.이미지에 있는 개체의 윤곽선을 찾음
6.윤곽선 근사치를 적용하여 해당 윤곽선이 직사각형인지를 확인해서 책인지 여부를 판단함